In [ ]:
import time
import requests
import pandas as pd
import yfinance as yf
from typing import List, Dict, Any

# --------------------------------------------------
# CONFIG
# --------------------------------------------------

# SEC company tickers JSON (ticker + CIK + exchange)
SEC_URL = "https://www.sec.gov/files/company_tickers_exchange.json"

# IMPORTANT: SEC requires a User-Agent with contact info
HEADERS = {
    "User-Agent": "your_email@example.com (for academic/Personal project)"
}

# How many tickers to process (set to None to use all)
MAX_TICKERS = None  # e.g. 3000 for quicker tests

# Sleep between Yahoo Finance calls (seconds)
YF_SLEEP = 0.2

# --------------------------------------------------
# STEP 1: Download US-listed universe from SEC
# --------------------------------------------------

import requests
import pandas as pd

SEC_URL = "https://www.sec.gov/files/company_tickers.json"  # or your chosen file


def load_sec_universe() -> pd.DataFrame:
    """
    Load a broad US-listed universe (incl. ADRs) from the SEC company_tickers JSON.

    Returns a DataFrame with columns: cik, ticker, name, exchange (if present).
    """
    headers = {
        "User-Agent": "your-email@example.com",  # SEC likes a UA with contact info
        "Accept": "application/json",
    }
    resp = requests.get(SEC_URL, headers=headers)
    resp.raise_for_status()
    raw = resp.json()

    # Figure out what the top-level structure is
    if isinstance(raw, dict):
        # Case 1: classic SEC format: {"0": {...}, "1": {...}, ...}
        # Case 2: {"data": [ {...}, {...}, ... ]}
        if "data" in raw and isinstance(raw["data"], list):
            iterable = raw["data"]
        else:
            iterable = raw.values()
    elif isinstance(raw, list):
        # Case 3: already a list of dicts
        iterable = raw
    else:
        raise TypeError(f"Unexpected SEC JSON structure: {type(raw)}")

    rows = []
    for entry in iterable:
        # Some safety checks to skip weird rows
        if not isinstance(entry, dict):
            continue
        if "ticker" not in entry or not entry["ticker"]:
            continue

        try:
            cik = int(entry.get("cik_str") or entry.get("cik") or 0)
        except (TypeError, ValueError):
            cik = None

        rows.append(
            {
                "cik": cik,
                "ticker": str(entry["ticker"]).upper(),
                "name": entry.get("title") or entry.get("name"),
                "exchange": entry.get("exchange"),
            }
        )

    df = pd.DataFrame(rows).dropna(subset=["ticker"])
    # Drop duplicates just in case
    df = df.drop_duplicates(subset=["ticker"]).reset_index(drop=True)

    print(f"Loaded {len(df)} SEC entries.")
    return df



# --------------------------------------------------
# STEP 2: Fetch market caps from Yahoo Finance
# --------------------------------------------------

def get_market_cap_yf(ticker: str) -> float | None:
    """
    Get market cap for a single ticker using yfinance.
    Returns None if not available / failed.
    """
    try:
        # use fast_info if available (newer yfinance), fall back to info
        t = yf.Ticker(ticker)

        mc = None
        # Try fast_info first
        try:
            fi = t.fast_info
            mc = getattr(fi, "market_cap", None)
        except Exception:
            pass

        # If fast_info didn't work, fall back to info dict
        if mc is None:
            info = t.info  # can be slow
            mc = info.get("marketCap")

        # Ensure numeric and positive
        if mc is None or not isinstance(mc, (int, float)) or mc <= 0:
            return None

        return float(mc)

    except Exception as e:
        # print(f"Error getting market cap for {ticker}: {e}")
        return None


def add_market_caps(df: pd.DataFrame) -> pd.DataFrame:
    """
    For each ticker in df, fetch market cap from Yahoo Finance.
    """
    tickers = df["ticker"].tolist()
    if MAX_TICKERS is not None:
        tickers = tickers[:MAX_TICKERS]

    market_caps = []
    total = len(tickers)
    print(f"Fetching market caps for {total} tickers via yfinance...")

    for i, ticker in enumerate(tickers, start=1):
        mc = get_market_cap_yf(ticker)
        market_caps.append(mc)

        if i % 50 == 0 or i == total:
            print(f"{i}/{total} tickers processed...")

        time.sleep(YF_SLEEP)

    df = df.iloc[: len(tickers)].copy()
    df["market_cap"] = market_caps
    return df


# --------------------------------------------------
# STEP 3: Build top-2000 by market cap
# --------------------------------------------------

def build_top_2000(output_csv: str = "top_2000_us_listed_by_market_cap.csv"):
    # 1) Universe from SEC
    base = load_sec_universe()

    # (Optional) Basic cleaning:
    # - Drop obvious non-common stocks if you want (e.g. some fund types),
    #   but since you *want* ADRs, we'll keep everything and let market cap sort it out.

    # 2) Add market caps from Yahoo
    with_caps = add_market_caps(base)

    # 3) Drop rows with missing market cap
    with_caps = with_caps[with_caps["market_cap"].notna()].copy()

    # 4) Sort by market cap descending and keep top 2000
    with_caps.sort_values("market_cap", ascending=False, inplace=True)
    top_2000 = with_caps.head(2000).reset_index(drop=True)

    print(f"\nRows with valid market cap: {len(with_caps)}")
    print(f"Top 2000 sample:\n{top_2000[['ticker','name','exchange','market_cap']].head(20)}")

    # 5) Save to CSV
    top_2000.to_csv(output_csv, index=False)
    print(f"\nSaved top 2000 list to: {output_csv}")


build_top_2000()



Loaded 10499 SEC entries.
Fetching market caps for 10499 tickers via yfinance...
50/10499 tickers processed...
100/10499 tickers processed...
150/10499 tickers processed...
200/10499 tickers processed...
250/10499 tickers processed...
300/10499 tickers processed...
350/10499 tickers processed...
400/10499 tickers processed...


$CHLSY: possibly delisted; no price data found  (period=5d)


450/10499 tickers processed...
500/10499 tickers processed...
550/10499 tickers processed...
600/10499 tickers processed...
650/10499 tickers processed...


$PMDIY: possibly delisted; no price data found  (period=5d)


700/10499 tickers processed...
750/10499 tickers processed...
800/10499 tickers processed...
850/10499 tickers processed...
900/10499 tickers processed...
950/10499 tickers processed...
1000/10499 tickers processed...
1050/10499 tickers processed...


...

10350/10499 tickers processed...


$ORIQU: possibly delisted; no price data found  (period=5d)
$SVCCU: possibly delisted; no price data found  (period=5d)
$AACBU: possibly delisted; no price data found  (period=5d)
$PAII-UN: possibly delisted; no price data found  (period=5d)


10400/10499 tickers processed...


$REECF: possibly delisted; no price data found  (period=5d)


10450/10499 tickers processed...
10499/10499 tickers processed...

Rows with valid market cap: 8355
Top 2000 sample:
    ticker                                       name exchange    market_cap
0     NVDA                                NVIDIA CORP     None  4.638834e+12
1     AAPL                                 Apple Inc.     None  4.057363e+12
2    GOOGL                              Alphabet Inc.     None  3.797272e+12
3     GOOG                              Alphabet Inc.     None  3.790565e+12
4     MSFT                             MICROSOFT CORP     None  3.625230e+12
5     AMZN                             AMAZON COM INC     None  2.485689e+12
6     META                       Meta Platforms, Inc.     None  1.671841e+12
7     AVGO                              Broadcom Inc.     None  1.669545e+12
8     TSLA                                Tesla, Inc.     None  1.580396e+12
9      TSM  TAIWAN SEMICONDUCTOR MANUFACTURING CO LTD     None  1.563290e+12
10   BMYMP                    BRISTO

In [ ]:
import datetime as dt
import numpy as np
import pandas as pd
import yfinance as yf

# -----------------------------
# 1. CONFIG
# -----------------------------
# Universe of tickers – you can replace this with S&P 500, IBD lists, etc.
tickers = pd.read_csv("top_2000_us_listed_by_market_cap.csv")
TICKERS = tickers['ticker'].to_list()

BENCHMARK = "SPY"   # for Relative Strength
LOOKBACK_DAYS = 365 # roughly 1 quarter
MIN_RS_PERCENTILE = 0.85  # top 15% RS

# -----------------------------
# 2. DATA DOWNLOAD
# -----------------------------
end = dt.date.today()
start = end - dt.timedelta(days=LOOKBACK_DAYS + 30)  # buffer for MAs

print(f"Downloading data from {start} to {end}...")

data = yf.download(TICKERS + [BENCHMARK], start=start, end=end, progress=False)["Close"]

# Drop any tickers with almost no data
data = data.dropna(how="all", axis=1)

# Separate benchmark series
bench = data[BENCHMARK].dropna()
prices = data.drop(columns=[BENCHMARK])

# -----------------------------
# 3. FEATURE ENGINEERING
# -----------------------------
def compute_ma(df, window):
    return df.rolling(window=window).mean()

ma50  = compute_ma(prices, 50)
ma150 = compute_ma(prices, 150)
ma200 = compute_ma(prices, 200)

latest_price = prices.iloc[-1]
latest_ma50  = ma50.iloc[-1]
latest_ma150 = ma150.iloc[-1]
latest_ma200 = ma200.iloc[-1]

# 52-week high/low over last ~252 trading days
N_52W = min(252, len(prices))
rolling_high_52w = prices.tail(N_52W).max()
rolling_low_52w  = prices.tail(N_52W).min()

# Moving-average slope: compare last value vs value 20 days ago
def ma_is_rising(ma_series, days=20):
    if len(ma_series.dropna()) < days + 1:
        return False
    return ma_series.iloc[-1] > ma_series.iloc[-1 - days]

ma200_rising = {
    ticker: ma_is_rising(ma200[ticker]) for ticker in prices.columns
}

# -----------------------------
# 4. RELATIVE STRENGTH (simple version)
# -----------------------------
# RS = stock 12-month return / benchmark 12-month return
stock_returns = prices.iloc[-1] / prices.iloc[0] - 1
bench_return  = bench.iloc[-1] / bench.iloc[0] - 1
rs_ratio = stock_returns / bench_return

# Convert to percentile rank (0–1)
rs_rank = rs_ratio.rank(pct=True)

# -----------------------------
# 5. MINERVINI FILTERS
# -----------------------------
rows = []
for ticker in prices.columns:
    if ticker not in latest_price.index:
        continue

    p   = latest_price[ticker]
    m50 = latest_ma50.get(ticker, np.nan)
    m150 = latest_ma150.get(ticker, np.nan)
    m200 = latest_ma200.get(ticker, np.nan)
    high_52w = rolling_high_52w[ticker]
    low_52w  = rolling_low_52w[ticker]

    if pd.isna(p) or pd.isna(m50) or pd.isna(m150) or pd.isna(m200):
        continue

    # Core Minervini Trend Template
    cond_price_above_mas = (p > m50) and (p > m150) and (p > m200)
    cond_ma_order        = (m50 > m150) and (m150 > m200)
    cond_ma200_rising    = ma200_rising.get(ticker, False)

    # Distance from 52-week low & high
    dist_from_low  = (p / low_52w - 1.0) if low_52w > 0 else np.nan
    dist_to_high   = (high_52w / p - 1.0) if p > 0 else np.nan

    cond_above_low = dist_from_low >= 0.30    # >= 30% above 52w low
    cond_near_high = dist_to_high <= 0.25     # within 25% of 52w high

    rs_val  = rs_rank.get(ticker, np.nan)
    cond_rs = rs_val >= MIN_RS_PERCENTILE

    passes = all([
        cond_price_above_mas,
        cond_ma_order,
        cond_ma200_rising,
        cond_above_low,
        cond_near_high,
        cond_rs
    ])

    rows.append({
        "ticker": ticker,
        "price": round(p, 2),
        "MA50": round(m50, 2),
        "MA150": round(m150, 2),
        "MA200": round(m200, 2),
        "52w_low": round(low_52w, 2),
        "52w_high": round(high_52w, 2),
        "%above_52w_low": round(dist_from_low * 100, 1),
        "%below_52w_high": round(dist_to_high * 100, 1),
        "RS_rank(0-1)": round(rs_val, 3),
        "MA200_rising": cond_ma200_rising,
        "passes_minervini": passes
    })

df = pd.DataFrame(rows).sort_values("RS_rank(0-1)", ascending=False)

print("\n=== Minervini-style Trend Template Results ===")
print(df.to_string(index=False))

print("\n=== Candidates that PASS all Minervini filters ===")
print(df[df["passes_minervini"]])


FileNotFoundError: [Errno 2] No such file or directory: 'top_2000_us_listed_by_market_cap.csv'

In [ ]:
final = df[(df.passes_minervini == True) & (df.ticker.str.len() <5 ) ][['ticker']]
final.to_csv("c:\\Users\Z370I5\Documents\minervini_screen.csv", index=False)

: 

In [ ]:
final

: 